In [1]:
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession, functions
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql.functions import explode
from pyspark.sql.functions import split
from pyspark.sql.types import *

In [2]:
sc = SparkContext()
ssc = StreamingContext(sc, 2)

In [3]:
lines = ssc.socketTextStream("localhost", 9000)
type(lines)

pyspark.streaming.dstream.DStream

In [4]:
window = lines.window(60)
type(window)

pyspark.streaming.dstream.DStream

In [5]:
def create_dict(df_int):
    num_srv = 100
    srv_dict = {}
    for i_srv in range(0, num_srv):
        # list of all "ip" for "sid"==i_srv
        ip_list = df_int[df_int["sid"] == i_srv]["ip"].tolist()
        # sort IP in the list
        ip_list.sort()
        #print(ip_list)

        ip_dict = {}
        # init with invalid value
        ip_prev = -1
        # go through all IP addresses for server 'i_srv'
        for i_ip in range(0, len(ip_list)):
            # skipp counting for already counted IP
            if ip_list[i_ip] != ip_prev:
                #print(ip_list.count(ip_list[i_ip]))
                # add key 'ip' with value 'count of requests' to dictionary 'ip_dict'
                ip_dict[ip_list[i_ip]] = ip_list.count(ip_list[i_ip])
                # remember last checked IP
                ip_prev = ip_list[i_ip]

        # add new key 'i_srv' in dictionarry with value 'ip_dict'
        if ip_dict != {}:
            srv_dict[i_srv]=ip_dict
#    print(srv_dict)
    return srv_dict

In [6]:
def similarity_a_b(array_a, array_b):
    summation = 0
    array_a_keys = list(array_a.keys())
    array_b_keys = list(array_b.keys())
    j = 0
    k = 0
    for m in range(0,max(len(array_a_keys),len(array_b_keys))):
        if (array_a_keys[j] < array_b_keys[k]):
#            print("j:", j, ", k:", k, ", value ip" + str(array_a_keys[j]) + ":", array_a.get(array_a_keys[j]), ", value ip" + str(array_b_keys[k]) + ":", array_b.get(array_b_keys[k]))
            j = j + 1
        elif (array_a_keys[j] > array_b_keys[k]):
#            print("j:", j, ", k:", k, ", value ip" + str(array_a_keys[j]) + ":", array_a.get(array_a_keys[j]), ", value ip" + str(array_b_keys[k]) + ":", array_b.get(array_b_keys[k]))
            k = k + 1
        elif (array_a_keys[j] == array_b_keys[k]):
#            print("j:", j, ", k:", k, ", value ip" + str(array_a_keys[j]) + ":", array_a.get(array_a_keys[j]), ", value ip" + str(array_b_keys[k]) + ":", array_b.get(array_b_keys[k]))
            product = array_a.get(array_a_keys[j]) * array_b.get(array_b_keys[k])
            summation = summation + product
#            print("product:", product, ", summation:", summation)
            j = j + 1
            k = k + 1
        if (j >= len(array_a_keys) or k >= len(array_b_keys)):
            break
#    print("Final summation:", summation)
    if summation > 3000:
        return 1
    else:
        return 0           

In [7]:
def similarity_all(thisdict, num_serv):
    count = 0
    for i in range(0,num_serv):
        for j in range(i+1,num_serv):
            count = count + similarity_a_b(thisdict.get(i), thisdict.get(j))
#    print(str(df_new.count()) + ",", count)
    return count

In [8]:
def process_stream(record, spark):
    if not record.isEmpty():
        df = spark.createDataFrame(record, StringType(), StringType())
        # implement all details here
#        print(df.printSchema)
#        print(type(df))
        df_pandas = df.toPandas()
        df_new = pd.DataFrame(columns=["sid", "ip"])
        df_new["sid"], df_new["ip"] = df_pandas["value"].str.split(",", 1).str
#        print(df_pandas)
        df_new["sid"] = df_new["sid"].astype(int)
        df_new["ip"] = df_new["ip"].astype(int)
#        print(type(df_new))
#        print(df_new.dtypes)

        srv_dict = create_dict(df_new)
        output = similarity_all(srv_dict, 100)
        print(output)

In [9]:
spark = SparkSession.builder.getOrCreate()
window.foreachRDD(lambda rdd: rdd.foreachPartition(process_stream(rdd, spark)))

In [10]:
ssc.start()
ssc.awaitTerminationOrTimeout(400)
#ssc.stop()

C:\ProgramData\Anaconda\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  if __name__ == '__main__':


0


Py4JJavaError: An error occurred while calling o22.awaitTerminationOrTimeout.
: org.apache.spark.SparkException: An exception was raised by Python:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda\lib\site-packages\pyspark\streaming\util.py", line 68, in call
    r = self.func(t, *rdds)
  File "C:\ProgramData\Anaconda\lib\site-packages\pyspark\streaming\dstream.py", line 161, in <lambda>
    func = lambda t, rdd: old_func(rdd)
  File "<ipython-input-9-16e2f33f0c2a>", line 2, in <lambda>
    window.foreachRDD(lambda rdd: rdd.foreachPartition(process_stream(rdd, spark)))
  File "C:\ProgramData\Anaconda\lib\site-packages\pyspark\rdd.py", line 879, in foreachPartition
    self.mapPartitions(func).count()  # Force evaluation
  File "C:\ProgramData\Anaconda\lib\site-packages\pyspark\rdd.py", line 1141, in count
    return self.mapPartitions(lambda i: [sum(1 for _ in i)]).sum()
  File "C:\ProgramData\Anaconda\lib\site-packages\pyspark\rdd.py", line 1132, in sum
    return se

In [ ]:
#ssc.stop()

C:\ProgramData\Anaconda\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  if __name__ == '__main__':


0


C:\ProgramData\Anaconda\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  if __name__ == '__main__':


0


C:\ProgramData\Anaconda\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  if __name__ == '__main__':


...

30


C:\ProgramData\Anaconda\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  if __name__ == '__main__':


31


C:\ProgramData\Anaconda\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  if __name__ == '__main__':


27
